In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import yfinance as yf
import pandas as pd
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Float,
    DateTime,
    Integer
)
import sqlite3
from typing import Optional, Literal
from datetime import datetime 
from dateutil.relativedelta import relativedelta

In [3]:
def get_ticker(company_name: str):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [4]:
def get_start_date(years: int = 10):
    today = datetime.today()
    return (today-relativedelta(years=years)).strftime("%Y-%m-%d")

In [5]:
def get_stock_data(
    ticker: str,
    start_date: Optional[str] = None) -> pd.DataFrame:
    """Gets the daily historical prices and volume for a ticker across a specified period"""
    
    if start_date is not None:
        return yf.download(
            ticker,
            start = start_date,
            end = datetime.today().strftime("%Y-%m-%d")
            )
    return yf.download(
        ticker,
        start = get_start_date(),
        end = datetime.today().strftime("%Y-%m-%d")
    )            
    
def connect_sql(db_path: str):
    conn = sqlite3.connect(db_path)
    sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
    cursor = conn.cursor()
    cursor.execute(sql_query)
    tables = cursor.fetchall()
    tables = [table[0] for table in tables]
    engine = create_engine(f"sqlite:///{db_path}")
    return engine, tables

## Start ingesting

In [17]:
def create_db_table(
    db_path: str, 
    company_name: str,
):
    engine, tables = connect_sql(db_path)
    metadata_obj = MetaData()
    table_name = get_ticker(company_name)
    table = Table(
        table_name,
        metadata_obj,
        Column("id", Integer, primary_key=True),
        Column("date", DateTime),
        Column("open", Float),
        Column("high", Float),
        Column("low", Float),
        Column("close", Float),
        Column("volume", Integer),
    )
    metadata_obj.create_all(engine)
    conn = sqlite3.connect(db_path)
    if table_name in tables: #update table
        start_date = pd.read_sql(
                f"""
                SELECT Date FROM {table_name} 
                ORDER by Date DESC
                LIMIT 1
                """,
                conn
            )['Date'].values[0].split(" ")[0]
        data = get_stock_data(ticker = table_name,
                              start_date = start_date).reset_index()
        data.to_sql(
            table_name,
            conn,
            if_exists='replace',
            index = False
        )
    else:
        data = get_stock_data(ticker = table_name).reset_index()
        data.to_sql(
            table_name,
            conn,
            index = False,
            if_exists = "replace"
        )

In [18]:
create_db_table(
    db_path = "../database/stocks.db",
    company_name='illumina'
)

[*********************100%%**********************]  1 of 1 completed


In [19]:
from tqdm.notebook import tqdm

for company in tqdm([
    "apple",
    "nvidia",
    "tesla",
    "microsoft",
    "amgen"
]):
    create_db_table(db_path="../database/stocks.db", company_name=company)

  0%|          | 0/5 [00:00<?, ?it/s]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [21]:
_, tables = connect_sql("../database/stocks.db")
tables

['ILMN', 'AAPL', 'NVDA', 'TSLA', 'MSFT', 'AMGN']

Test connection

In [32]:
conn = sqlite3.connect("../database/stocks.db")
pd.read_sql(
    """
    SELECT * FROM AMGN 
    ORDER BY Date DESC
    LIMIT 10
    """,
    conn
)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-07-26 00:00:00,334.559998,338.940002,333.279999,334.850006,334.850006,1515200
1,2024-07-25 00:00:00,335.989990,346.850006,333.019989,334.299988,334.299988,2435700
2,2024-07-24 00:00:00,333.250000,337.179993,330.339996,335.609985,335.609985,2459300
3,2024-07-23 00:00:00,334.359985,337.040009,331.309998,333.239990,333.239990,1458200
4,2024-07-22 00:00:00,333.230011,337.089996,332.470001,335.970001,335.970001,1733700
5,2024-07-19 00:00:00,333.649994,333.980011,328.429993,331.290009,331.290009,1848700
6,2024-07-18 00:00:00,334.200012,338.980011,330.670013,330.970001,330.970001,2228400
7,2024-07-17 00:00:00,330.000000,336.609985,328.600006,335.670013,335.670013,1933200
8,2024-07-16 00:00:00,330.459991,335.910004,328.000000,333.140015,333.140015,1879400
9,2024-07-15 00:00:00,331.880005,332.809998,327.869995,330.149994,330.149994,1688800
